In [9]:
import cv2
import numpy as np
from scipy import interpolate

def deg2rad(d):
  return float(d) * np.pi / 180

def rotate_image(old_image):
  (old_height, old_width, _) = old_image.shape
  M = cv2.getRotationMatrix2D(((old_width - 1) / 2., (old_height - 1) / 2.), 270, 1)
  rotated = cv2.warpAffine(old_image, M, (old_width, old_height))
  return rotated

def xrotation(th):
  c = np.cos(th)
  s = np.sin(th)
  return np.array([[1, 0, 0], [0, c, s], [0, -s, c]])

def yrotation(th):
  c = np.cos(th)
  s = np.sin(th)
  return np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])


def render_image_np(theta0, phi0, fov_h, fov_v, width, img, pitch):
    """
    theta0 is  pitch
    phi0 is yaw
    render view at (pitch, yaw) with fov_h by fov_v
    width is the number of horizontal pixels in the view
    """
    m = np.dot(yrotation(phi0), xrotation(theta0))  # original
    
    # Huan
    m = np.dot(yrotation(phi0), np.dot(xrotation(pitch), xrotation(theta0))  # original
#     m = np.dot(yrotation(phi0), xrotation(theta0))  # original
    

    #   (base_height, base_width, _) = img.shape
    base_height = img.shape[0]
    base_width = img.shape[1]

    height = int(width * np.tan(fov_v / 2) / np.tan(fov_h / 2))

    new_img = np.zeros((height, width, 3), np.uint8)

    DI = np.ones((height * width, 3), np.int)
    trans = np.array([[2.*np.tan(fov_h / 2) / float(width), 0., -np.tan(fov_h / 2)],
                    [0., -2.*np.tan(fov_v / 2) / float(height), np.tan(fov_v / 2)]])

    xx, yy = np.meshgrid(np.arange(width), np.arange(height))

    DI[:, 0] = xx.reshape(height * width)
    DI[:, 1] = yy.reshape(height * width)

    v = np.ones((height * width, 3), np.float)

    v[:, :2] = np.dot(DI, trans.T)
    v = np.dot(v, m.T)

    diag = np.sqrt(v[:, 2] ** 2 + v[:, 0] ** 2)
    theta = np.pi / 2 - np.arctan2(v[:, 1], diag)
    phi = np.arctan2(v[:, 0], v[:, 2]) + np.pi

    ey = np.rint(theta * base_height / np.pi).astype(np.int)
    ex = np.rint(phi * base_width / (2 * np.pi)).astype(np.int)

    ex[ex >= base_width] = base_width - 1
    ey[ey >= base_height] = base_height - 1  

    new_img[DI[:, 1], DI[:, 0]] = img[ey, ex]
    return new_img

def render_image_np_1c(theta0, phi0, fov_h, fov_v, width, img):
    """
    theta0 is pitch
    phi0 is yaw
    render view at (pitch, yaw) with fov_h by fov_v
    width is the number of horizontal pixels in the view
    """
    m = np.dot(yrotation(phi0), xrotation(theta0))

    #   (base_height, base_width, _) = img.shape
    base_height = img.shape[0]
    base_width = img.shape[1]

    height = int(width * np.tan(fov_v / 2) / np.tan(fov_h / 2) + 0.5)

    new_img = np.zeros((height, width), np.float)

    DI = np.ones((height * width, 3), np.int)
    trans = np.array([[2.*np.tan(fov_h / 2) / float(width), 0., -np.tan(fov_h / 2)],
                    [0., -2.*np.tan(fov_v / 2) / float(height), np.tan(fov_v / 2)]])

    xx, yy = np.meshgrid(np.arange(width), np.arange(height))

    DI[:, 0] = xx.reshape(height * width)
    DI[:, 1] = yy.reshape(height * width)

    v = np.ones((height * width, 3), np.float)

    v[:, :2] = np.dot(DI, trans.T)
    v = np.dot(v, m.T)

    diag = np.sqrt(v[:, 2] ** 2 + v[:, 0] ** 2)
    theta = np.pi / 2 - np.arctan2(v[:, 1], diag)
    phi = np.arctan2(v[:, 0], v[:, 2]) + np.pi
    
    grid_col = np.linspace(0, 512, 512)
    grid_row = np.linspace(0, 256, 256)
    
    interp = interpolate.interp2d(grid_col, grid_row, img[:,:, 0], kind='linear') # 'cubic' will distord the distance.


    ey = np.array(theta * base_height / np.pi).astype(np.float)
    ex = np.array(phi * base_width / (2 * np.pi)).astype(np.float)
    
    ey = ey.reshape((height, width))
    ex = ex.reshape((height, width))
    
    print(ex)
    
    for r in range(height):
        for c in range(width):
            new_img[r][c] = interp(ex[r][c], ey[r][c])[0]

#     ex[ex >= base_width] = base_width - 1
#     ey[ey >= base_height] = base_height - 1  

#     new_img[DI[:, 1], DI[:, 0]] = img[ey, ex]
    return new_img


def equi_to_cube(face_size, img):
  """
  given an equirectangular spherical image, project it onto standard cube
  """
  cube_img_h = face_size * 3
  cube_img_w = face_size * 2
  cube_img = np.zeros((cube_img_h, cube_img_w, 3), np.uint8)

  ii = render_image_np(np.pi / 2, np.pi, \
                      np.pi / 2, np.pi / 2, \
                      face_size, img)
#   cv2.imwrite('g_top.jpg', ii)

  cube_img[:cube_img_h / 3, cube_img_w / 2:] = ii.copy()

  ii = render_image_np(0, 0, \
                      np.pi / 2, np.pi / 2, \
                      face_size, img)
#   cv2.imwrite('g_front.jpg', ii)

  cube_img[cube_img_h / 3:cube_img_h * 2 / 3, :cube_img_w / 2] = rotate_image(ii).copy()

  ii = render_image_np(0, np.pi / 2, \
                      np.pi / 2, np.pi / 2, \
                      face_size, img)
#   cv2.imwrite('g_right.jpg', ii)

  cube_img[cube_img_h * 2 / 3:, :cube_img_w / 2] = rotate_image(ii).copy()

  ii = render_image_np(0, np.pi, \
                      np.pi / 2, np.pi / 2, \
                      face_size, img)
#   cv2.imwrite('g_back.jpg', ii)

  cube_img[cube_img_h / 3:cube_img_h * 2 / 3, cube_img_w / 2:] = ii.copy()

  ii = render_image_np(0, np.pi * 3 / 2, \
                      np.pi / 2, np.pi / 2, \
                      face_size, img)
#   cv2.imwrite('g_left.jpg', ii)

  cube_img[:cube_img_h / 3, :cube_img_w / 2] = rotate_image(ii).copy()

  ii = render_image_np(-np.pi / 2, np.pi, \
                      np.pi / 2, np.pi / 2, \
                      face_size, img)
#   cv2.imwrite('g_bottom.jpg', ii)

  cube_img[cube_img_h * 2 / 3:, cube_img_w / 2:] = ii.copy()

#   cv2.imwrite('g_cube.jpg', cube_img)
  return cube_img

# if __name__ == '__main__':
#     file = r'K:\OneDrive_NJIT\OneDrive - NJIT\Research\sidewalk\Essex_test\panos\Dg0LGX581Uu5ooKrLlvQGA.jpg'
 
    
#     img = cv2.imread(file)
#     face_size = 1024

#     yaw = 90
#     pitch = 0
#     fov_h = 90*748/1024*768/684
#     fov_v = 90

#     rimg = render_image_np(deg2rad(pitch), deg2rad(yaw), \
#                           deg2rad(fov_v), deg2rad(fov_h), \
#                           face_size, img)

#     cv2.imwrite('rendered_image_%d_%d.bmp' % (pitch, yaw), rimg)

In [3]:
from math import *
from PIL import Image
file = r'O:\OneDrive_NJIT\OneDrive - NJIT\Research\sidewalk\Essex_test\panos\Dg0LGX581Uu5ooKrLlvQGA.tif'

# file = r'K:\OneDrive_NJIT\OneDrive - NJIT\Research\sidewalk\Essex_test\panos\Dg0LGX581Uu5ooKrLlvQGA.jpg'

img = cv2.imread(file)
img = Image.open(file)
img = np.array(img)

img = np.expand_dims(img, 3)


# print(img)
face_size = 1024
w  = 1024
h = 768
face_size = w


yaw = -90
pitch = 0

fov_v = 90
fov_h = atan((h*tan((radians(fov_v)/2))/w))*2
fov_h = degrees(fov_h)
print('fov_h: ',fov_h)

rimg = render_image_np_1c(deg2rad(pitch), deg2rad(yaw), \
                      deg2rad(fov_v), deg2rad(fov_h), \
                      face_size, img)
cv2.imwrite('rendered_image_%d_%d.bmp' % (pitch, yaw), rimg)

d:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  # This is added back by InteractiveShellApp.init_path()


fov_h:  73.73979529168803
[[ 64.          64.07965523  64.1594662  ... 191.76056681 191.8405338
  191.92034477]
 [ 64.          64.07965523  64.1594662  ... 191.76056681 191.8405338
  191.92034477]
 [ 64.          64.07965523  64.1594662  ... 191.76056681 191.8405338
  191.92034477]
 ...
 [ 64.          64.07965523  64.1594662  ... 191.76056681 191.8405338
  191.92034477]
 [ 64.          64.07965523  64.1594662  ... 191.76056681 191.8405338
  191.92034477]
 [ 64.          64.07965523  64.1594662  ... 191.76056681 191.8405338
  191.92034477]]


True

In [65]:
def render_image_np(theta0, phi0, fov_h, fov_v, width, img, pano_pitch):
    """
    theta0 is  pitch
    phi0 is yaw
    render view at (pitch, yaw) with fov_h by fov_v
    width is the number of horizontal pixels in the view
    """
#      m = np.dot(yrotation(phi0), xrotation(theta0))  # original
    
    # Huan
    m = xrotation(pitch)
    m = np.dot(m, np.dot(yrotation(phi0), xrotation(theta0)))
#     m = np.dot(np.dot(yrotation(phi0), xrotation(theta0)), m)

    
#     m = np.dot(yrotation(phi0), np.dot(xrotation(pitch), xrotation(theta0)))  # original
#     m = np.dot(yrotation(phi0), xrotation(theta0))  # original
    

    #   (base_height, base_width, _) = img.shape
    base_height = img.shape[0]
    base_width = img.shape[1]

    height = int(width * np.tan(fov_v / 2) / np.tan(fov_h / 2))

    new_img = np.zeros((height, width, 3), np.uint8)

    DI = np.ones((height * width, 3), np.int)
    trans = np.array([[2.*np.tan(fov_h / 2) / float(width), 0., -np.tan(fov_h / 2)],
                    [0., -2.*np.tan(fov_v / 2) / float(height), np.tan(fov_v / 2)]])

    xx, yy = np.meshgrid(np.arange(width), np.arange(height))

    DI[:, 0] = xx.reshape(height * width)
    DI[:, 1] = yy.reshape(height * width)

    v = np.ones((height * width, 3), np.float)

    v[:, :2] = np.dot(DI, trans.T)
    v = np.dot(v, m.T)

    diag = np.sqrt(v[:, 2] ** 2 + v[:, 0] ** 2)
    theta = np.pi / 2 - np.arctan2(v[:, 1], diag)
    phi = np.arctan2(v[:, 0], v[:, 2]) + np.pi

    ey = np.rint(theta * base_height / np.pi).astype(np.int)
    ex = np.rint(phi * base_width / (2 * np.pi)).astype(np.int)

    ex[ex >= base_width] = base_width - 1
    ey[ey >= base_height] = base_height - 1  

    new_img[DI[:, 1], DI[:, 0]] = img[ey, ex]
    return new_img


In [74]:
from math import *
w  = 1024
h = 768

panoId  = r'qAYd9QiwszKOAKE7TGmfuQ'
pano_pitch = 7.93000000000
# pano_pitch = 5.1
pitch = 0
yaw = -90
heading = yaw + 118.23000000000


file = r'O:\OneDrive_NJIT\OneDrive - NJIT\Research\sidewalk\Essex_test\panos\Dg0LGX581Uu5ooKrLlvQGA.jpg'
file = f'O:\OneDrive_NJIT\OneDrive - NJIT\Research\sidewalk\Essex_test\panos\{panoId}.jpg'
url = f'https://geo1.ggpht.com/cbk?cb_client=maps_sv.tactile&authuser=0&output=thumbnail&panoid={panoId}&h={h}&w={w}&pitch={pitch}&yaw={heading}'

print("url: ", url)

img = cv2.imread(file)
# print(img)
print(type(img))
 

face_size = w


# yaw = -90
# pitch = 0

fov_h = 90
fov_v = atan((h*tan((radians(fov_h)/2))/w))*2
fov_v = degrees(fov_v)
print('fov_h: ',fov_v)



rimg = render_image_np(deg2rad(pitch), deg2rad(yaw), \
                      deg2rad(fov_h), deg2rad(fov_v), \
                      face_size, img, deg2rad(pano_pitch))
cv2.imwrite('%s_%d_%d.bmp' % (panoId, pitch, yaw), rimg)

url:  https://geo1.ggpht.com/cbk?cb_client=maps_sv.tactile&authuser=0&output=thumbnail&panoid=qAYd9QiwszKOAKE7TGmfuQ&h=768&w=1024&pitch=0&yaw=28.230000000000004
<class 'numpy.ndarray'>
fov_h:  73.73979529168803


True

In [59]:
new_img = np.zeros((10, 20), np.uint8)

In [60]:
new_img.shape

(10, 20)

In [75]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

ModuleNotFoundError: No module named 'pydrive'